In [7]:
%pip -q install google-genai
!pip install -q google-adk

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

    # Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
def agente_buscador_inspiracao(estilo_musical, tema, data_de_hoje):
    buscador = Agent(
        name="agente_buscador_inspiracao",
        model="gemini-2.0-flash",
        instruction=(
            "Você é um assistente especializado em buscar referências musicais. "
            "Use a ferramenta de busca do Google (google_search) para encontrar músicas populares "
            "e recentes dentro do estilo musical e tema indicados. "
            "Liste até 5 músicas relevantes, informando nome da música, artista, ano de lançamento "
            "e principais temas abordados na letra. "
            "As músicas devem ser dos últimos 12 meses a partir da data informada."
        ),
        description="Agente que busca músicas populares para inspiração na criação de letras.",
        tools=[google_search]
    )

    entrada_do_agente = f"Estilo musical: {estilo_musical}\nTema: {tema}\nData de hoje: {data_de_hoje}"

    referencias = call_agent(buscador, entrada_do_agente)
    return referencias

def agente_planejador_letra(estilo_musical, referencias_musicais):
    planejador = Agent(
        name="agente_planejador_letra",
        model="gemini-2.0-flash",
        instruction=(
            "Você é um planejador criativo especializado em composição musical, "
            f"com profundo conhecimento do estilo {estilo_musical} especialmente das músicas que viralizam e ficam no ouvido das pessoas. "
            f"Com base na lista de músicas populares e recentes encontradas, você deve criar um plano detalhado para a letra de uma nova música {estilo_musical}. "
            f"Destacar os principais temas que tocam o público {estilo_musical} atual, como amor, sofrência, festa, vida no interior, superação, entre outros. "
            "Identifique emoções fortes e palavras ou frases de efeito que costumam grudar na cabeça dos ouvintes. "
            "Além disso, considere elementos estruturais que fazem a música ser fácil de cantar junto, como refrões repetitivos e linguagem simples. "
            "Use a ferramenta de busca do Google (google_search) para aprofundar informações sobre tendências sertanejas e temas populares. "
            "Ao final, selecione o tema mais relevante para a composição e descreva a estrutura geral que a letra deve ter (versos, refrão, ponte, etc.), "
            "com foco em engajamento e viralização."
        ),
        description=f"Agente que planeja letras de músicas {estilo_musical} virais com base nas referências encontradas.",
        tools=[google_search]
    )

    entrada_do_agente = f"Estilo musical: {estilo_musical}\nReferências musicais: {referencias_musicais}"

    plano_letra = call_agent(planejador, entrada_do_agente)
    return plano_letra

def agente_redator_letra(estilo_musical, plano_de_letra):
    redator = Agent(
        name="agente_redator_letra",
        model="gemini-2.0-flash",
        instruction=(
            "Você é um compositor criativo especializado em letras de músicas populares. "
            "Com base no estilo musical e no plano fornecido, escreva um rascunho de letra de música "
            "que seja envolvente, coerente com o tema, e adequado ao gênero indicado. "
            "Use linguagem poética e emocional, e respeite a estrutura típica do estilo musical."
        ),
        description="Agente redator de letras de músicas populares."
    )

    entrada_do_agente = f"Estilo musical: {estilo_musical}\nPlano de letra: {plano_de_letra}"

    rascunho_letra = call_agent(redator, entrada_do_agente)
    return rascunho_letra

def agente_revisor_letra(estilo_musical, rascunho_letra):
    revisor = Agent(
        name="agente_revisor_letra",
        model="gemini-2.0-flash",
        instruction=(
            "Você é um editor e revisor especializado em letras de músicas. "
            "Seu trabalho é analisar profundamente o rascunho da letra fornecida, verificando: "
            "- Clareza das ideias e narrativa coerente;\n"
            "- Impacto emocional e conexão com o estilo musical;\n"
            "- Estrutura harmônica e fluidez dos versos;\n"
            "- Uso de figuras de linguagem e originalidade.\n\n"
            "Revise a letra e, se fizer alterações, apresente a letra completa revisada. "
            "Caso faça sugestões, já incorpore essas mudanças no texto final. "
            "Se estiver perfeita, apenas destaque os pontos positivos."
        ),
        description="Agente revisor de letras de música."
    )

    entrada_do_agente = (
        f"Estilo musical: {estilo_musical}\n\n"
        f"Rascunho da letra:\n{rascunho_letra}\n\n"
        "Revise a letra, sugira melhorias e, se necessário, apresente a letra revisada completa."
    )

    texto_revisado = call_agent(revisor, entrada_do_agente)
    print(texto_revisado)  # Para visualizar a saída
    return texto_revisado

from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Letras Musicais com 4 Agentes 🚀")

# --- Obter o Estilo Musical do Usuário ---
estilo_musical = input("❓ Por favor, digite o ESTILO MUSICAL para o qual deseja criar a letra (ex: sertanejo, pop, rock): ").strip()
tema_musca = input("❓ Por favor, digite o TEMA que deseja para a letra: ").strip()

if not estilo_musical:
    print("Você esqueceu de digitar o estilo musical!")
if not tema_musca:
    print("Você esqueceu de digitar o tema da letra!")
else:
    print(f"Maravilha! Vamos criar uma letra para o estilo {estilo_musical}")

    # Passo 1: Buscar referências musicais atuais para o estilo escolhido
    referencias_musicais = agente_buscador_inspiracao(estilo_musical, "Cicatrizes do amor" ,  data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador de referências musicais) ---\n")
    display(to_markdown(referencias_musicais))
    print("--------------------------------------------------------------")

    # Passo 2: Planejar a letra com base nas referências encontradas
    plano_letra = agente_planejador_letra(estilo_musical, referencias_musicais)
    print("\n--- 📝 Resultado do Agente 2 (Planejador da letra) ---\n")
    display(to_markdown(plano_letra))
    print("--------------------------------------------------------------")

    # Passo 3: Gerar o rascunho da letra
    rascunho_letra = agente_redator_letra(estilo_musical, plano_letra)
    print("\n--- 📝 Resultado do Agente 3 (Redator da letra) ---\n")
    display(to_markdown(rascunho_letra))
    print("--------------------------------------------------------------")

    # Passo 4: Revisar e aprimorar a letra criada
    letra_final = agente_revisor_letra(estilo_musical, plano_letra)
    print("\n--- 📝 Resultado do Agente 4 (Revisor da letra) ---\n")
    display(to_markdown(letra_final))
    print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Letras Musicais com 4 Agentes 🚀
❓ Por favor, digite o ESTILO MUSICAL para o qual deseja criar a letra (ex: sertanejo, pop, rock): sertanejo - pop rock
❓ Por favor, digite o TEMA que deseja para a letra: Dias de paz
Maravilha! Vamos criar uma letra para o estilo sertanejo - pop rock

--- 📝 Resultado do Agente 1 (Buscador de referências musicais) ---



> Para te ajudar a encontrar inspiração, vou buscar músicas recentes no estilo sertanejo com influências de pop rock, que abordem o tema das "cicatrizes do amor". Veja a seguir as músicas encontradas:
> 
> 
> Com base nas minhas buscas, aqui estão algumas sugestões de músicas que podem te inspirar, dentro do estilo e tema solicitados:
> 
> 1.  **"A Gente Fez Amor - Ao Vivo" - Gusttavo Lima (2024):** Essa música fala sobre a intensidade de um amor que marcou, mesmo que tenha chegado ao fim, deixando lembranças.
> 
> 2. **"3 Batidas - Ao Vivo" - Guilherme & Benuto (2024):** A canção explora a persistência das memórias de um relacionamento, mesmo após a separação, e como o coração reage a essas lembranças.
> 
> 3.  **"Infarto" - Diego & Victor Hugo (2024):** A música aborda a dor de um amor perdido de forma intensa, descrevendo o sofrimento como um "infarto" emocional.
> 
> 4.  **"Vai ter que superar" - (Artista não especificado nas fontes, mas popular em 2025):** A letra fala sobre a necessidade de seguir em frente após um término, focando na superação das dificuldades emocionais.
> 
> 5.  **"Esqueci o caminho de casa" - (Artista não especificado nas fontes, mas popular em 2025):** Retrata a desorientação e a tristeza após o fim de um relacionamento, onde a pessoa se sente perdida e busca alívio na bebida.
> 
> Espero que essas sugestões te inspirem na criação de suas letras!
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador da letra) ---



> Olá! Que bom que você me trouxe essas referências. Com base nelas, vou criar um plano para uma nova música sertanejo-pop rock sobre "cicatrizes do amor", focando em viralização e engajamento.
> 
> **1. Análise das Referências:**
> 
> *   **Temas em Comum:** As músicas compartilham a dor do amor perdido, a dificuldade de superar o fim do relacionamento e a intensidade das lembranças. A "sofrência" é um elemento central, mas com uma pegada pop que facilita a identificação do público.
> *   **Emoções Fortes:** Dor, saudade, desorientação, superação (ou a necessidade dela).
> *   **Palavras/Frases de Efeito:** "A gente fez amor" (lembrança forte), "3 batidas" (ritmo do coração), "Infarto" (metáfora da dor), "Vai ter que superar" (mensagem de força), "Esqueci o caminho de casa" (desorientação).
> *   **Estrutura:** Refrões repetitivos, linguagem simples, melodias que grudam.
> 
> **2. Pesquisa de Tendências:**
> 
> Para garantir que a música esteja alinhada com o que está em alta no sertanejo-pop rock em 2025, vou usar o Google para pesquisar:
> 
> 
> **3. Insights da Pesquisa:**
> 
> *   **Artistas em Destaque:** Gusttavo Lima, Guilherme & Benuto, Diego & Victor Hugo, Simone Mendes, Zé Neto & Cristiano continuam relevantes em 2025.
> *   **Temas Populares:** Sofrência, superação, recaídas (não conseguir esquecer o ex), a vida no interior (referências à fazenda, bebida).
> *   **Elementos Virais:** Músicas com letras fáceis de cantar junto, refrões marcantes, melodias que viralizam no TikTok (coreografias, trends).
> 
> **4. Definição do Tema:**
> 
> Com base nas referências e nas tendências, o tema mais relevante para a música é: **A recaída na sofrência, misturada com a superação forçada.**
> 
> **Justificativa:**
> 
> *   **Conecta-se com as "Cicatrizes do Amor":** A recaída mostra que as cicatrizes ainda doem.
> *   **É Atual:** O tema da superação forçada (fingir que superou, mas ainda sofre) está em alta.
> *   **Gera Identificação:** Quem nunca tentou superar um amor e falhou?
> *   **Permite um Refrão Forte:** A contradição entre "superar" e "recaír" é ideal para um refrão que grude.
> 
> **5. Estrutura da Letra:**
> 
> *   **Verso 1:** Descrever a vida da pessoa "superada" - saindo, bebendo, "vivendo a vida". Mas com um tom de atuação, de quem está tentando convencer a si mesmo.
> *   **Verso 2:** A recaída. Um encontro inesperado, uma mensagem, algo que faz a pessoa lembrar do ex e toda a "superação" ir por água abaixo.
> *   **Pré-Refrão:** A luta interna. A pessoa tentando resistir, mas sentindo que vai perder a batalha. Criar tensão para o refrão.
> *   **Refrão:** A explosão da sofrência. Uma frase curta e impactante que resume a recaída. Exemplo:
>     *   "Eu superei, juro que superei / Mas o replay do seu beijo me pegou de novo"
>     *   "Tava na night, beijando outra boca / Mas seu fantasma beijou em dobro"
> *   **Ponte:** Uma reflexão mais profunda sobre as "cicatrizes do amor". A pessoa admite que não superou de verdade e que talvez nunca supere.
> *   **Repetição:** Repetir o refrão várias vezes (a chave para viralizar).
> *   **Final:** Terminar com uma frase que deixe o ouvinte pensando ("E agora, o que eu faço?").
> 
> **6. Elementos de Engajamento e Viralização:**
> 
> *   **Linguagem:** Usar gírias e expressões atuais (ex: "tô na bad", "sextou").
> *   **Refrão:** Criar um refrão chiclete, com palavras simples e melodia fácil de cantar.
> *   **TikTok:** Pensar em um trecho da música que possa viralizar em challenges e trends.
> *   **Clipe:** Criar um clipe com uma história visualmente interessante e que dialogue com a letra.
> 
> **7. Título da Música:**
> 
> Algumas sugestões de títulos que capturam a essência da música:
> 
> *   "Supera Que Recai"
> *   "Beijo Fantasma"
> *   "Replay do Beijo"
> *   "Cicatriz Que Não Fecha"
> *   "Ato Falho da Superação"
> 
> Este é um plano detalhado para criar uma música sertanejo-pop rock com o tema "cicatrizes do amor" e foco em viralização. Espero que te ajude a compor uma canção incrível!
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator da letra) ---



> Ok, com base no plano detalhado que você forneceu, aqui está um rascunho da letra da música sertanejo-pop rock sobre "cicatrizes do amor", focando na recaída na sofrência e na superação forçada:
> 
> **Título:** Supera Que Recai
> 
> **(Intro - Batida de sertanejo com elementos de pop rock, guitarra leve)**
> 
> **(Verso 1)**
> Sextou, tô no jet, copo na mão
> Fazendo story rindo à toa, mó diversão
> Pra legenda: "Vida que segue, sem caô"
> Mas o filtro esconde a saudade, meu amor
> 
> **(Verso 2)**
> Achei que a playlist do Spotify resolvia
> Que uns beijos roubados na night curavam a ferida
> Aí tocou "nossa música" no paredão
> E o castelo de superação foi pro chão
> 
> **(Pré-Refrão)**
> Tentei fugir, juro que tentei
> Mas a lembrança do teu beijo me invade, não sei porquê
> Meu coração, um GPS desatualizado
> Só sabe o caminho de volta pro teu pecado
> 
> **(Refrão)**
> Eu superei, juro que superei
> Mas o replay do seu beijo me pegou de novo
> Eu bloqueei, deletei, ignorei
> Mas teu fantasma beijou em dobro, que sufoco!
> 
> **(Verso 3)**
> Diz que tá bem, que arrumou outro alguém
> Posta foto feliz, mas a gente sabe, né?
> Que por dentro a saudade ainda faz morada
> E que essa superação é só fachada
> 
> **(Pré-Refrão)**
> Tentei fugir, juro que tentei
> Mas a lembrança do teu beijo me invade, não sei porquê
> Meu coração, um GPS desatualizado
> Só sabe o caminho de volta pro teu pecado
> 
> **(Refrão)**
> Eu superei, juro que superei
> Mas o replay do seu beijo me pegou de novo
> Eu bloqueei, deletei, ignorei
> Mas teu fantasma beijou em dobro, que sufoco!
> 
> **(Ponte)**
> Cicatrizes do amor, rabiscos na alma
> A gente tenta apagar, mas a tinta não se acalma
> Será que um dia essa dor vai ter fim?
> Ou vou viver pra sempre nesse vai e vem?
> 
> **(Refrão)**
> Eu superei, juro que superei
> Mas o replay do seu beijo me pegou de novo
> Eu bloqueei, deletei, ignorei
> Mas teu fantasma beijou em dobro, que sufoco!
> 
> **(Final)**
> E agora, o que eu faço com essa recaída?
> Será que volto correndo pros teus braços, ou me livro dessa vida?
> (Fade out com a batida e um toque de guitarra)
> 
> **Notas:**
> 
> *   A linguagem é simples e direta, com algumas gírias ("caô", "jet").
> *   O refrão é repetitivo e fácil de cantar, ideal para viralizar.
> *   A melodia deve ser animada e dançante, com uma pegada pop rock.
> *   O clipe pode mostrar a protagonista tentando "superar" em festas e baladas, mas sempre sendo atormentada pelas lembranças do ex.
> *   O trecho do "fantasma beijou em dobro" pode ser usado em challenges do TikTok com efeitos visuais.
> *   O título "Supera Que Recai" resume bem a temática da música e é fácil de lembrar.
> 
> Espero que goste do rascunho! Se precisar de mais alguma coisa, é só avisar.
> 


--------------------------------------------------------------
O plano para a criação da música sertanejo-pop rock está bem estruturado e detalhado, demonstrando um bom entendimento das tendências atuais e dos elementos que podem impulsionar a viralização. A análise das referências e a pesquisa de tendências são sólidas, e a definição do tema ("A recaída na sofrência, misturada com a superação forçada") é pertinente e promissora.

**Pontos Positivos:**

*   **Planejamento Detalhado:** O plano aborda todos os aspectos importantes da criação da música, desde a análise de referências até a definição do título e elementos de engajamento.
*   **Foco na Viralização:** A preocupação em criar um refrão chiclete, pensar em trends para o TikTok e criar um clipe visualmente interessante demonstra um bom entendimento de como impulsionar a viralização da música.
*   **Tema Relevante e Atual:** O tema da recaída na sofrência, misturada com a superação forçada, é relevante e atual, e tem potencial pa

> O plano para a criação da música sertanejo-pop rock está bem estruturado e detalhado, demonstrando um bom entendimento das tendências atuais e dos elementos que podem impulsionar a viralização. A análise das referências e a pesquisa de tendências são sólidas, e a definição do tema ("A recaída na sofrência, misturada com a superação forçada") é pertinente e promissora.
> 
> **Pontos Positivos:**
> 
> *   **Planejamento Detalhado:** O plano aborda todos os aspectos importantes da criação da música, desde a análise de referências até a definição do título e elementos de engajamento.
> *   **Foco na Viralização:** A preocupação em criar um refrão chiclete, pensar em trends para o TikTok e criar um clipe visualmente interessante demonstra um bom entendimento de como impulsionar a viralização da música.
> *   **Tema Relevante e Atual:** O tema da recaída na sofrência, misturada com a superação forçada, é relevante e atual, e tem potencial para gerar identificação com o público.
> *   **Estrutura da Letra Bem Definida:** A estrutura da letra proposta é coerente e bem pensada, com um fluxo narrativo que conduz o ouvinte através das emoções da música.
> 
> **Sugestões e Letra Revisada:**
> 
> Com base no plano, desenvolvi uma letra completa, incorporando alguns dos elementos sugeridos e adicionando alguns toques pessoais para dar mais força e originalidade à música.
> 
> **Título:** Beijo de Replay
> 
> **Letra:**
> 
> (Verso 1)
> Sextou, liguei o carro, som no talo
> A galera me esperando no point
> Copo na mão, sorriso no rosto
> Dizendo pra mim mesma: "De você, eu tô free, enfim"
> Maquiagem, batom, look arrasador
> No espelho, uma selfie pra provar
> Que a vida segue, que a fila andou
> Que o seu amor já não me faz mais chorar
> 
> (Verso 2)
> Mas, de repente, toca aquela canção
> A nossa música, no meio do salão
> E a saudade vem, me pega de surpresa
> Um flash, um déjà vu, uma fraqueza
> Te vejo entrando, lindo como sempre
> Meu mundo para, a gravidade aumenta
> Um sorriso torto, um olhar inocente
> E a minha "superação" vai pra estaca zero, meu Deus, que tormento
> 
> (Pré-Refrão)
> Eu tento disfarçar, finjo que não ligo
> Mas o meu corpo trai, ele te segue
> A razão grita "não", o coração suplica
> E eu me afogo nesse mar de recaída
> 
> (Refrão)
> Eu superei, juro que superei
> Mas o replay do seu beijo me pegou de novo
> Eu jurei que não, que nunca mais ia
> Mas o replay do seu beijo me viciou de novo
> Beijo de replay, tortura que vicía
> Beijo de replay, saudade que não alivia
> 
> (Ponte)
> Dizem que o tempo cura, que as cicatrizes fecham
> Mas as suas digitais ainda estão cravadas em mim
> E, mesmo querendo fugir, eu volto sempre pra você
> Porque, no fundo, a gente nunca esquece um grande amor, né?
> 
> (Refrão)
> Eu superei, juro que superei
> Mas o replay do seu beijo me pegou de novo
> Eu jurei que não, que nunca mais ia
> Mas o replay do seu beijo me viciou de novo
> Beijo de replay, tortura que vicía
> Beijo de replay, saudade que não alivia
> 
> (Final)
> E agora, o que eu faço com esse beijo de replay?
> Será que eu fujo ou me entrego de uma vez?
> 
> **Observações:**
> 
> *   **Refrão Marcante:** O refrão foi construído com frases curtas e impactantes, fáceis de cantar e memorizar.
> *   **Linguagem Atual:** A letra utiliza gírias e expressões atuais, como "sextou", "tô free", "point", para criar uma identificação com o público jovem.
> *   **TikTok:** O trecho do refrão ("Eu superei, juro que superei / Mas o replay do seu beijo me pegou de novo") tem potencial para viralizar no TikTok, com challenges e trends relacionados à superação e recaída.
> *   **Título:** O título "Beijo de Replay" é sugestivo e impactante, e resume bem a essência da música.
> 
> Acredito que essa letra revisada, combinada com o plano detalhado que você criou, tem o potencial de se tornar um grande sucesso no sertanejo-pop rock.


--------------------------------------------------------------
